In [ ]:
from autots import AutoTS
import pandas as pd
import matplotlib.pyplot as plt

long=False
df = pd.read_csv("BASEo.csv", index_col=0, parse_dates=True, dayfirst=True)
df = df.iloc[:,:4]
df['Año'] = pd. DatetimeIndex(df.index).year
df['Mes'] = pd. DatetimeIndex(df.index).month
df.tail()

In [ ]:
train = df.loc[(df['Tipo Dia'] == 7)]
train = train[:-24]
test = df.iloc[-24:]
train = train.iloc[:,:4]
train.head()

In [ ]:
regr_train, regr_fcst = create_lagged_regressor(
    train['Clima GBA'],
    forecast_length=24,
    backfill='bfill',
    fill_na='ffill',
    frequency="H"
)

# remove the first forecast_length rows (because those are lost in regressor)
train = train.iloc[24:]
regr_train = regr_train.iloc[24:]

In [ ]:
model = AutoTS(model_list= "default", forecast_length=24, verbose=1, frequency="H",
    ensemble= "simple",
    transformer_list="all",
    max_generations=2,
    num_validations=2,
    validation_method="seasonal 168")
model.fit(train, future_regressor=regr_train)

In [ ]:
prediction = model.predict()
forecasts_df = prediction.forecast

# accuracy of all tried model results
model_results = model.results()
# and aggregated from cross validation
validation_results = model.results("Validation")
forecasts_df.head(24)

In [ ]:
forecasts_df.to_excel("forecast.xlsx")

In [ ]:
validation_results.sort_values("Score")

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=forecasts_df.index, y=test["Energia BSAS"], name='Consumo'))
fig.add_trace(go.Scatter(x=forecasts_df.index, y=forecasts_df["Energia BSAS"], name = 'Forecast'))
fig.show()

In [ ]:
example_filename = "modelos.csv"
model.export_template(example_filename, models='best',
					  n=3, max_per_model_class=3)